In [125]:
#matching (K2/2MASS) + UCAC4(=Sampedro)
import catalog_match as cm
import os
import sys
import numpy as np


In [126]:
#x.loadcatalog('/home/ekaterina/Documents/Matching_Catalogs','_Sampedro_cluster_members_query.csv',(2,22),',', 1,[('UCAC4','U10'),('2MASSYN','U10')],debug=False)
dt=np.dtype(np.float64)
dtstr=np.dtype(np.unicode_, 10)
cat=np.genfromtxt('/home/ekaterina/Documents/Matching_Catalogs/cats/M67_Sampedro_cluster_members_query.csv' ,usecols=(2,22),delimiter='\t', skip_header=1,dtype=[('UCAC4','U10'),('2MASSYN',dt)])
samp_twomassyn=cat['2MASSYN'].tolist()
samp_twomassyn = [str(r) for r in samp_twomassyn]
samp_ucac4=cat['UCAC4'].tolist()
samp_ucac4 = [str(r) for r in samp_ucac4]
print(samp_ucac4[:10])
indices = [i for i, x in enumerate(samp_twomassyn) if x == -9999.99]
samp_twomassyn_with_ucac4=[i for j, i in enumerate(samp_twomassyn) if j not in indices]
ucac4_samp=[i for j, i in enumerate(samp_ucac4) if j not in indices]


['508-049159', '508-049161', '508-049164', '508-049170', '508-049171', '508-049173', '508-049174', '508-049178', '508-049180', '508-049181']


In [127]:
k2=np.genfromtxt('/home/ekaterina/Documents/Matching_Catalogs/cats/M67_k2_search.txt' ,usecols=(19,20),delimiter='\t', skip_header=2,dtype=[('UCAC4','U10'),('2MASS','U16')])

twomass_k2=k2['2MASS'].tolist()
twomass_k2 = [str(r) for r in twomass_k2]
ucac4_k2=k2['UCAC4'].tolist()
ucac4_k2 = [str(r) for r in ucac4_k2]
print(len(ucac4_k2))

indices = [i for i, x in enumerate(ucac4_k2) if x == '']
ucac4_k2=[i for j, i in enumerate(ucac4_k2) if j not in indices]
twomass_k2=[i for j, i in enumerate(twomass_k2) if j not in indices]
print(len(ucac4_k2))#habe ucac4 id
indices = [i for i, x in enumerate(twomass_k2) if x == '']
ucac4_k2=[i for j, i in enumerate(ucac4_k2) if j not in indices]
twomass_k2=[i for j, i in enumerate(twomass_k2) if j not in indices]
print(len(ucac4_k2))#habe ucac4 id und 2mass


752
358
358


In [128]:

before=len(ucac4_samp)
#ucac4_k2=[1,4,5,7,9,2,3]
#ucac4_samp=[6,5,4,8,11,1,0]
match=[]
for item in ucac4_k2:
    if item in ucac4_samp:
        match.append(item)
        ucac4_samp.remove(item)
print(before-len(ucac4_samp))

158


In [68]:
import pandas as pd
df = pd.read_csv('/home/ekaterina/Documents/Matching_Catalogs/cats/K2EPIC_Sampedro_PanSTARRS.csv')
                # names=['UCAC4','DEJ2000','RAJ2000','angDist1','angDist2',]),

df = df[(df['angDist.1'] < 0.5) & (df.angDist < 0.5)]

df = df[(df.P_M1 > 0.8) & (df.P_M2_join > 0.8) & (df.P_M2_kinem > 0.8) & (df.P_M3 > 0.8) ]

grouped = df.groupby('UCAC4')
df = df[df.angDist.isin(grouped.angDist.min().values)] 

sel = df[['id','gmag.1','rmag.1','imag.1','zmag.1','ymag','Jmag','Hmag','Kmag','RAJ2000','DEJ2000']]
sel = sel.rename(index=str, columns={'id':'EPIC','gmag.1':'gmag','rmag.1':'rmag','imag.1':'imag','zmag.1':'zmag','Jmag':'J',
                  'Hmag':'H','Kmag':'K'})
sel.head()

,EPIC,gmag,rmag,imag,zmag,ymag,J,H,K,RAJ2000,DEJ2000
0,247113403,16.763,15.729,15.370,15.1762,15.0083,13.946,13.479,13.436,71.3571,18.7975
1,247112847,NaN,NaN,NaN,15.8890,15.6921,14.477,13.869,13.668,71.4222,18.7922
2,247111108,NaN,NaN,NaN,15.4767,15.2543,14.054,13.439,13.215,71.4537,18.7758
3,247113519,16.412,15.310,14.926,14.6765,14.5298,13.479,13.004,12.797,71.4595,18.7986
4,247113466,NaN,NaN,NaN,14.5751,14.4431,13.394,13.005,12.868,71.4712,18.7980


In [70]:
cluster = 'NGC_1647'
sel.to_csv('share/{0}/{0}_parameter.csv'.format(cluster))

In [ ]:
0